In [1]:
import os
import sys
import json
import torch
import pickle
import logging
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings(action='ignore', message='Too many lattice symmetries was found')
import random
import time

from pymatgen.ext.matproj import MPRester

import pickle
import logging

# IOs
from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.phase_diagram import PhaseDiagram, GrandPotentialPhaseDiagram
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixEntry,MultiEntry
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.core.periodic_table import Element, Species
from pymatgen.core.composition import Composition
from pymatgen.analysis.structure_analyzer import oxide_type
from pymatgen.entries.computed_entries import ComputedEntry, ComputedStructureEntry
from pymatgen.analysis.interface_reactions import InterfacialReactivity, GrandPotentialInterfacialReactivity
from pymatgen.entries.entry_tools import EntrySet

# To remove m3gnet error
import tensorflow as tf
import warnings
tf.get_logger().setLevel(logging.ERROR)

# House code
sys.path.append('./')
from relaxer import TrajectoryObserver, M3gnetRelaxer, ChgnetRelaxer, MaceRelaxer

2024-06-23 09:14:10.936010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 09:14:10.936052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 09:14:10.937690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 09:14:10.947979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 09:14:12.457353: W tensorflow/compiler/tf2

In [2]:
def read_json(fjson):
    
    with open(fjson) as f:
        return json.load(f)


def write_json(d, fjson):
    
    with open(fjson, 'w') as f:
        json.dump(d, f)

    return

In [2]:
mpr = MPRester(api_key="dPcAQJZ6y1NZidGuvTerIPPFXHtsOb3E")

def has_common_element(list1, list2):
    return not set(list1).isdisjoint(list2)

def get_GGA_entry(vasp_data, entry_id, compat=MaterialsProject2020Compatibility()):
    lattice = vasp_data['final_structure'].lattice.matrix
    species = [specie for specie in vasp_data['final_structure'].species]
    coords = [site.frac_coords for site in vasp_data['final_structure'].sites]
    structure = Structure(lattice=lattice, species=species, coords=coords)
    gga_entry = ComputedStructureEntry(structure=structure,
                                       energy=vasp_data['final_energy'],
                                       entry_id=entry_id,
                                       composition=structure.composition.remove_charges(),
                                       parameters={'run_type': vasp_data['run_type'],
                                                   'potcar_symbols': vasp_data['potcar_symbols'],
                                                   'hubbards': vasp_data['hubbards']
                                                   }
                                       )

    if compat is not None:
        gga_entry = compat.process_entry(gga_entry)
    if gga_entry is None:
        print('Entry is None')
        exit()
    
    return gga_entry

def get_ehull(mpr, vasp_entry, mp_entries, compatibility):
    
    mp_entries = compatibility.process_entries(mp_entries)

    # phase diagram
    PD = PhaseDiagram(mp_entries + [vasp_entry])
    decomp_info = PD.get_decomp_and_e_above_hull(vasp_entry, allow_negative=True)
    e_above_hull = decomp_info[1]

    return e_above_hull

/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module named 'phonopy'
No module named 'phonopy'


In [3]:
def get_test_structure(mpid='mp-510462'):
    
    mp_data = mpr.get_entry_by_material_id(material_id=[test_id])
    test_structure = mp_data[1].structure
    
    return test_structure


def get_chemsys_mpdata(structure):
    
    U_els = {'Co': 3.32, 'Cr': 3.7, 'Fe': 5.3, 'Mn': 3.9,
             'Mo': 4.38, 'Ni': 6.2, 'V': 4.2, 'W': 6.2}
    
    species = []
    potcar_spec = []

    for i in set(test_structure.species):
        species.append(i.name)

    chemical_space = '-'.join(species)
    mp_data = mpr.get_entries_in_chemsys(chemical_space)
    
    hubbards = {}
    if has_common_element(list(U_els.keys()), species):
        for specie in species:
            if specie in U_els.keys():
                hubbards[specie] = U_els[specie]
            else:
                hubbards[specie] = 0

    for d in mp_data:
        for j in d.parameters['potcar_spec']:
            if not j in potcar_spec:
                potcar_spec.append(j)
        if len(potcar_spec) == len(species):
            break

    return mp_data, potcar_spec, hubbards

def get_relaxation_result(structure):
        
    mp_data, potcar_spec, hubbards = get_chemsys_mpdata(structure)
    
    atoms = structure.to_ase_atoms()
    relaxer = MaceRelaxer()
    result = relaxer.relax(atoms, fmax=0.5)
    result['parameters'] = mp_data[0].parameters
    result['parameters']['potcar_spec'] = potcar_spec
    result['parameters']['hubbards'] = hubbards
    
    lattice = result['final_structure'].lattice.matrix
    species = [specie for specie in result['final_structure'].species]
    coords = [site.frac_coords for site in result['final_structure'].sites]
    structure = Structure(lattice=lattice, species=species, coords=coords)

    gga_entry = ComputedStructureEntry(structure=structure,
                                       energy=result['final_energy'],
                                       entry_id='llm_generation',
                                       composition=structure.composition.remove_charges(),
                                       parameters=result['parameters']
                                       )

    compat = MaterialsProject2020Compatibility()
    gga_entry = compat.process_entry(gga_entry)
    ehull = get_ehull(mpr, gga_entry, mp_data, compat)
    
    return result, ehull

In [4]:
test_structure = Structure.from_file('../test/test_POSCAR')

In [5]:
result, ehull = get_relaxation_result(test_structure)

Retrieving ThermoDoc documents: 100%|██████████| 76/76 [00:00<00:00, 1250067.07it/s]
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


No dtype selected, switching to float64 to match model dtype.
MACE will run on cpu
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 20:19:12      -45.464003*       0.0760


/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-505634-GGA (Cs2Te3). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-1228983-GGA (CsZn8). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-570071-GGA (Dy2Zn17). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-1193674-GGA (Dy6Zn2

In [8]:
result['trajectory'].atoms_trajectory[1]

MSONAtoms(symbols='Cs2Dy2Zn2Te6', pbc=True, cell=[[4.430704, 0.0, 0.0], [-2.215352, 8.689916, 0.0], [0.0, 0.0, 11.747494]], constraint=FixAtoms(indices=[]), calculator=MACECalculator(...))

In [9]:
aaa = AseAtomsAdaptor()

In [11]:
aaa = AseAtomsAdaptor()
aaa.get_structure(result['trajectory'].atoms_trajectory[1])

Structure Summary
Lattice
    abc : 4.430704 8.967855070804836 11.747494
 angles : 90.0 90.0 104.30198229452905
 volume : 452.3072484465264
      A : 4.430704 0.0 0.0
      B : -2.215352 8.689916 0.0
      C : 0.0 0.0 11.747494
    pbc : True True True
PeriodicSite: Cs (2.215, 4.279, 8.811) [0.7462, 0.4924, 0.75]
PeriodicSite: Cs (0.0, 4.411, 2.937) [0.2538, 0.5076, 0.25]
PeriodicSite: Dy (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Dy (0.0, 0.0, 5.874) [0.0, 0.0, 0.5]
PeriodicSite: Zn (0.0, 8.03, 8.811) [0.462, 0.924, 0.75]
PeriodicSite: Zn (2.215, 0.66, 2.937) [0.538, 0.07595, 0.25]
PeriodicSite: Te (0.0, 6.626, 11.07) [0.3813, 0.7625, 0.9423]
PeriodicSite: Te (2.215, 2.064, 0.6781) [0.6187, 0.2375, 0.05772]
PeriodicSite: Te (2.215, 2.064, 5.196) [0.6187, 0.2375, 0.4423]
PeriodicSite: Te (0.0, 6.626, 6.552) [0.3813, 0.7625, 0.5577]
PeriodicSite: Te (0.0, 1.01, 8.811) [0.0581, 0.1162, 0.75]
PeriodicSite: Te (2.215, 7.68, 2.937) [0.9419, 0.8838, 0.25]

In [118]:
get_relaxation_result(test_structure)

Retrieving ThermoDoc documents: 100%|██████████| 76/76 [00:00<00:00, 1207451.15it/s]


No dtype selected, switching to float64 to match model dtype.
MACE will run on cpu
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:35:45      -45.464003*       0.0760


({'final_structure': Structure Summary
  Lattice
      abc : 4.430704 8.967855070804836 11.747494
   angles : 90.0 90.0 104.30198229452905
   volume : 452.3072484465264
        A : 4.430704 0.0 0.0
        B : -2.215352 8.689916 0.0
        C : 0.0 0.0 11.747494
      pbc : True True True
  PeriodicSite: Cs (2.215, 4.279, 8.811) [0.7462, 0.4924, 0.75]
  PeriodicSite: Cs (0.0, 4.411, 2.937) [0.2538, 0.5076, 0.25]
  PeriodicSite: Dy (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
  PeriodicSite: Dy (0.0, 0.0, 5.874) [0.0, 0.0, 0.5]
  PeriodicSite: Zn (0.0, 8.03, 8.811) [0.462, 0.924, 0.75]
  PeriodicSite: Zn (2.215, 0.66, 2.937) [0.538, 0.07595, 0.25]
  PeriodicSite: Te (0.0, 6.626, 11.07) [0.3813, 0.7625, 0.9423]
  PeriodicSite: Te (2.215, 2.064, 0.6781) [0.6187, 0.2375, 0.05772]
  PeriodicSite: Te (2.215, 2.064, 5.196) [0.6187, 0.2375, 0.4423]
  PeriodicSite: Te (0.0, 6.626, 6.552) [0.3813, 0.7625, 0.5577]
  PeriodicSite: Te (0.0, 1.01, 8.811) [0.0581, 0.1162, 0.75]
  PeriodicSite: Te (2.215, 7.68, 2.

In [98]:
mp_data, potcar_spec= get_chemsys_mpdata(test_structure)

Retrieving ThermoDoc documents: 100%|██████████| 76/76 [00:00<00:00, 1024974.61it/s]


In [100]:
test_atoms = test_structure.to_ase_atoms()

In [101]:
relaxer = MaceRelaxer()

No dtype selected, switching to float64 to match model dtype.
MACE will run on cpu


In [102]:
result = relaxer.relax(test_atoms, fmax=0.05)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 18:30:12      -45.464003*       0.0760
FIRE:    1 18:30:12      -45.464246*       0.0725
FIRE:    2 18:30:12      -45.464685*       0.0656
FIRE:    3 18:30:13      -45.465237*       0.0557
FIRE:    4 18:30:13      -45.465796*       0.0433


In [103]:
result

dict_keys(['final_structure', 'final_energy', 'trajectory'])

In [104]:
result['parameters'] = mp_data[1].parameters

In [107]:
result['parameters']

{'potcar_spec': [{'titel': 'PAW_PBE Dy_3 06Sep2000',
   'hash': 'd4a05220ab0a2d4c03a76872ea724a1e'}],
 'is_hubbard': False,
 'hubbards': {},
 'run_type': 'GGA'}

In [33]:
lattice = result['final_structure'].lattice.matrix
species = [specie for specie in result['final_structure'].species]
coords = [site.frac_coords for site in result['final_structure'].sites]
structure = Structure(lattice=lattice, species=species, coords=coords)

gga_entry = ComputedStructureEntry(structure=structure,
                                   energy=result['final_energy'],
                                   entry_id='tester-12',
                                   composition=structure.composition.remove_charges(),
                                   parameters=result['parameters']
                                   )

# compat = MaterialsProject2020Compatibility()

# if compat is not None:
#     gga_entry = compat.process_entry(gga_entry)

In [34]:
gga_entry

tester-12 ComputedStructureEntry - Cs2 Dy2 Zn2 Te6 (CsDyZnTe3)
Energy (Uncorrected)     = -45.4658  eV (-3.7888  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -45.4658  eV (-3.7888  eV/atom)
Energy Adjustments:
  None
Parameters:
  potcar_spec            = [{'titel': 'PAW_PBE Cs_sv 08Apr2002', 'hash': '096b53a7d80cc0086976bcda50d536e5'}, {'titel': 'PAW_PBE Dy_3 06Sep2000', 'hash': 'd4a05220ab0a2d4c03a76872ea724a1e'}, {'titel': 'PAW_PBE Zn 06Sep2000', 'hash': 'e35ee27f8483a63bb68dbc236a343af3'}, {'titel': 'PAW_PBE Te 08Apr2002', 'hash': '72719856e22fb1d3032df6f96d98a0f2'}]
  is_hubbard             = False
  hubbards               = {}
  run_type               = GGA
Data:

In [35]:
gga_entry = compat.process_entry(gga_entry)

In [54]:
ehull(mpr, gga_entry, MaterialsProject2020Compatibility())

Retrieving ThermoDoc documents: 100%|██████████| 76/76 [00:00<00:00, 684049.58it/s]
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-570071-GGA (Dy2Zn17). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-1193674-GGA (Dy6Zn23). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry mp-1104161-GGA (Dy3Zn11). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
/global/homes/y/yychoi/.conda/envs/mace/lib/python3.9/site-packages/pymatgen/entries/compatibility

0.007546575649457932